In [1]:
!pip install azure-storage-blob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 6.2 MB/s eta 0:00:00


In [2]:
#!pip install azure-storage-blob

import pandas as pd
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

connection_string = "DefaultEndpointsProtocol=https;AccountName=mloblistorage;AccountKey=u+SGxSxdN+0KtUq2qwRS2zD+Ul4hBnt+nhAzPok53QggOuXa9+5V3VU8NDHiGmG2zN+ghQ7V+AtY+ASt8kPFcA==;EndpointSuffix=core.windows.net"
container_name ="mloblistoragecontainer"
blob_name = "properties_tienda_inglesa.csv"

blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

blob_client = container_client.get_blob_client(blob_name)
csv_data = blob_client.download_blob().content_as_text()

data = StringIO(csv_data)
df = pd.read_csv(data, header=None, names=['id','image_urls','images','link','product_name','property_type','source','url'])
#id,image_urls,images,link,product_name,property_type,source,url

valid_categories=["WAREHOUSE",
             "DRINKS",
            "CLEANING",
             "FROZEN",
            "TECHNOLOGY",
             "PERFUMERY"]


In [3]:
df = df_filtrado = df.query('property_type in @valid_categories')
df = df[~df['id'].duplicated()]
df['md5']=''

<ipython-input-3-214b070c72a5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['md5']=''


In [4]:
def get_image_from_blob(blob_client):
    stream = io.BytesIO()
    download_stream = blob_client.download_blob()
    stream.write(download_stream.readall())

    stream.seek(0)
    image = Image.open(stream)
    return image


In [5]:
def show_image(blob_client, blob_name):

    image = get_image_from_blob(blob_client, blob_name)

    plt.imshow(image)
    plt.show()


In [6]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import io
from PIL import Image
import matplotlib.pyplot as plt

try:
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_name = 'full/0094e2445afde938bd97477f21149e43101f1ebf.jpg'
    blob_client = blob_service_client.get_blob_client(container_name, blob_name)

    show_image(blob_client, blob_name)

except Exception as ex:
    print('Exception:')
    print(ex)

Exception:
get_image_from_blob() takes 1 positional argument but 2 were given


In [7]:
def calcular_md5(imagen):
    md5_hash = hashlib.md5(imagen).hexdigest()
    return md5_hash

In [8]:
IMAGE_SIZE=32

In [ ]:

import ast
import hashlib

image_ids=[]
images =[]
categories=[]
count=1;
for index, row in df.iterrows():
    lista_json = row['images']
    b=ast.literal_eval(lista_json[1:-1])
    path =  b.get("path")

    blob_client = blob_service_client.get_blob_client(container_name, path)
    imagen=get_image_from_blob(blob_client).resize((IMAGE_SIZE,IMAGE_SIZE))
    md5=calcular_md5(imagen.tobytes())
    if(md5 not in df['md5'].values):
      images.append(imagen)
      df.at[index,'md5'] = md5

      image_ids.append(row['id'])
      categories.append(row['property_type'])
      if count%100==0:
        print(f"Load {count} images of {df.shape[0]}")
      count+=1



Load 100 images of 6229
Load 200 images of 6229
Load 300 images of 6229
Load 400 images of 6229
Load 500 images of 6229
Load 600 images of 6229
Load 700 images of 6229
Load 800 images of 6229
Load 900 images of 6229
Load 1000 images of 6229
Load 1100 images of 6229
Load 1200 images of 6229
Load 1300 images of 6229
Load 1400 images of 6229
Load 1500 images of 6229
Load 1600 images of 6229
Load 1700 images of 6229
Load 1800 images of 6229
Load 1900 images of 6229
Load 2000 images of 6229
Load 2100 images of 6229
Load 2200 images of 6229
Load 2300 images of 6229


In [ ]:
df

In [ ]:
import tensorflow as tf
from PIL import Image

def convert_image_to_example(image, image_id,category):
    feature = {
        'id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[str(image_id).encode()])),
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image.tobytes()])),
        'property_type': tf.train.Feature(bytes_list=tf.train.BytesList(value=[category.encode()])),
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def save_images_to_tfrecords(images, image_ids,categories ,output_file):
    with tf.io.TFRecordWriter(output_file) as writer:
        for image, category in zip(zip(images, image_ids),categories):
            tf_example = convert_image_to_example(image[0],image[1], category)
            writer.write(tf_example)


# Guardar las imagenes y sus IDs en un archivo TFRecords
save_images_to_tfrecords(images, image_ids, categories ,'images.tfrecords')



In [ ]:
# Función para leer un archivo TFRecords y recuperar las imágenes y sus IDs
def read_images_from_tfrecords(input_file):
    raw_dataset = tf.data.TFRecordDataset(input_file)
    count=0
    for _ in raw_dataset:
      count+=1

    print(count)
    feature_description = {
        'id': tf.io.FixedLenFeature([], tf.string),
        'image': tf.io.FixedLenFeature([], tf.string),
        'property_type': tf.io.FixedLenFeature([], tf.string),
    }


    images = []
    image_ids = []
    property_types =[]
    for raw_record in raw_dataset:
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        image_bytes = example.features.feature['image'].bytes_list.value[0]
        image = Image.frombytes('RGB', (32, 32), image_bytes)

        image_id = example.features.feature['id'].bytes_list.value[0].decode()
        property_type = example.features.feature['property_type'].bytes_list.value[0].decode()

        images.append(image)
        property_types.append(property_type)


    return images, image_ids, property_types




# Leer el archivo TFRecords y recuperar las imágenes y sus IDs
loaded_images, loaded_image_ids, property_types = read_images_from_tfrecords('images.tfrecords')

In [ ]:
print(loaded_images)